In [1]:
import pandas as pd
import numpy as np
import warnings

# --- 0. CONFIGURAÇÕES E FUNÇÕES AUXILIARES ---
print("Iniciando o Pipeline de Limpeza Completo...")
print("Serão processados 10 arquivos-fonte (16 arquivos físicos).")
print("Isto pode demorar alguns minutos...")

# Ignorar os avisos de "FutureWarning"
warnings.simplefilter(action='ignore', category=FutureWarning)

# Função para detectar o header do Arquivo 5 (Sua função)
def detectar_header_municipio(arquivo, pd_instance):
    tmp = pd_instance.read_excel(arquivo, header=None, nrows=15)
    for i in range(len(tmp)):
        linha = tmp.iloc[i].astype(str).str.lower().tolist()
        if any("munic" in x for x in linha):
            return i
    raise Exception(f"Não foi possível detectar a linha do cabeçalho com 'Município' no arquivo {arquivo}.")

# Função para "achatar" cabeçalhos (Sua função)
def flatten_ibge_cols(multi_cols):
    col_names = []
    for c in multi_cols:
        if not isinstance(c, tuple): c = (c,)
        partes = [str(x).strip() for x in c]
        partes = [p for p in partes if not p.startswith("Unnamed") and p != 'nan' and not p.startswith("Variável")]
        if len(partes) == 1:
            col_names.append(partes[0])
            continue
        nome = " - ".join(partes)
        nome = " ".join(nome.split())
        col_names.append(nome)
    return col_names

# Lista para guardar os DataFrames finais
dataframes_finais = {}

try:
    # --- ARQUIVO 1: IDADE (Tabela 9514) ---
    print("\n[1/10] Processando Arquivo 1: Idade (Tabela 9514)...")
    caminho_idade = '01_IBGE_Censo2022_Idade_Municipios.xlsx'
    df_idade_limpo = pd.read_excel(caminho_idade, header=5)
    df_idade_limpo.rename(columns={'Unnamed: 0': 'Município'}, inplace=True)
    df_idade_limpo = df_idade_limpo.drop(0).reset_index(drop=True)
    colunas_piramide = [
        'Município',
        '0 a 4 anos', '5 a 9 anos', '10 a 14 anos', '15 a 19 anos',
        '20 a 24 anos', '25 a 29 anos', '30 a 34 anos', '35 a 39 anos',
        '40 a 44 anos', '45 a 49 anos', '50 a 54 anos', '55 a 59 anos',
        '60 a 64 anos', '65 a 69 anos', '70 a 74 anos', '75 a 79 anos',
        '80 a 84 anos', '85 a 89 anos', '90 a 94 anos', '95 a 99 anos',
        '100 anos ou mais'
    ]
    df_idade_limpo = df_idade_limpo[colunas_piramide].copy()
    cols_numericas_idade = colunas_piramide[1:]
    df_idade_limpo[cols_numericas_idade] = df_idade_limpo[cols_numericas_idade].replace('-', 0)
    df_idade_limpo[cols_numericas_idade] = df_idade_limpo[cols_numericas_idade].apply(pd.to_numeric, errors='coerce').fillna(0)
    col_jovens = ['0 a 4 anos', '5 a 9 anos', '10 a 14 anos']
    col_idosos = [
        '60 a 64 anos', '65 a 69 anos', '70 a 74 anos', '75 a 79 anos',
        '80 a 84 anos', '85 a 89 anos', '90 a 94 anos', '95 a 99 anos',
        '100 anos ou mais'
    ]
    df_idade_limpo['Jovens (0-14)'] = df_idade_limpo[col_jovens].sum(axis=1)
    df_idade_limpo['Idosos (60+)'] = df_idade_limpo[col_idosos].sum(axis=1)
    df_idade_limpo['Indice_Envelhecimento'] = np.where(
        df_idade_limpo['Jovens (0-14)'] > 0,
        (df_idade_limpo['Idosos (60+)'] / df_idade_limpo['Jovens (0-14)']) * 100,
        0
    ).round(2)
    dataframes_finais['Idade'] = df_idade_limpo.copy()
    print("-> SUCESSO: Arquivo 1 (Idade) limpo.")


    # --- ARQUIVO 2: DOMICÍLIOS (Tabela 9879) ---
    print("\n[2/10] Processando Arquivo 2: Domicílios (Tabela 9879)...")
    caminho_domicilios = '02_IBGE_Censo2022_Domicilios_Municipios.xlsx'

    # Carregar o df_domi_bruto (como você diagnosticou)
    df_domi_bruto = pd.read_excel(caminho_domicilios, header=[1, 3, 4, 5])

    df_domi_bruto.columns = [
        'Município', 'Cor_Raca', 'Grupo_Idade',
        'Total_Domicilios', 'Unipessoal', 'Nuclear'
    ]

    df_domi_bruto['Município'] = df_domi_bruto['Município'].ffill()
    df_domi_bruto['Cor_Raca'] = df_domi_bruto['Cor_Raca'].ffill()

    # Usar df_domi_limpo (como você diagnosticou)
    df_domi_limpo = df_domi_bruto.drop(0).reset_index(drop=True)
    df_domi_limpo['Município'] = df_domi_limpo['Município'].str.strip()
    df_domi_limpo['Cor_Raca'] = df_domi_limpo['Cor_Raca'].str.strip()
    df_domi_limpo['Grupo_Idade'] = df_domi_limpo['Grupo_Idade'].str.strip()

    cols_numericas_domi = ['Total_Domicilios', 'Unipessoal', 'Nuclear']
    df_domi_limpo[cols_numericas_domi] = df_domi_limpo[cols_numericas_domi].replace('-', 0)
    df_domi_limpo[cols_numericas_domi] = df_domi_limpo[cols_numericas_domi].apply(pd.to_numeric, errors='coerce').fillna(0)

    # Salvar a tabela completa (para análises futuras de raça/idade)
    dataframes_finais['Domicilios_Completo'] = df_domi_limpo.copy()

    # O "Patch Anti-Duplicação" que você criou
    df_total_raca = df_domi_limpo[df_domi_limpo['Cor_Raca'] == 'Total'].copy()
    df_total_raca = df_total_raca[df_total_raca['Grupo_Idade'].isin(['Total', '60 anos ou mais'])]
    df_total_raca = df_total_raca.sort_values(by=['Município', 'Grupo_Idade']).drop_duplicates(subset=['Município', 'Grupo_Idade'], keep='first')

    df_total_idade = df_total_raca[df_total_raca['Grupo_Idade'] == 'Total'].copy()
    df_total_idade['%_Unipessoais'] = (df_total_idade['Unipessoal'] / df_total_idade['Total_Domicilios'] * 100).round(2)

    df_60mais = df_total_raca[df_total_raca['Grupo_Idade'] == '60 anos ou mais'].copy()
    df_60mais['%_Unipessoais_60+'] = (df_60mais['Unipessoal'] / df_60mais['Total_Domicilios'] * 100).round(2)

    df_final_domicilios = pd.merge(
        df_total_idade[['Município', 'Total_Domicilios', 'Unipessoal', '%_Unipessoais']],
        df_60mais[['Município', 'Total_Domicilios', 'Unipessoal', '%_Unipessoais_60+']],
        on='Município',
        suffixes=('_Geral', '_60+')
    )
    dataframes_finais['Domicilios_Agg'] = df_final_domicilios.copy()
    print("-> SUCESSO: Arquivo 2 (Domicílios) limpo.")


    # --- ARQUIVO 3: RENDA (Tabela 10291) ---
    print("\n[3/10] Processando Arquivo 3: Renda (Tabela 10291)...")
    caminho_renda = '03_IBGE_Censo2022_Renda_Municipios.xlsx'
    df_renda_limpo = pd.read_excel(caminho_renda, skiprows=5, header=None)
    df_renda_limpo.columns = [
        'Município', 'Grupo_Idade', 'Total', 'Branca', 'Preta',
        'Amarela', 'Parda', 'Indígena'
    ]
    df_renda_limpo['Município'] = df_renda_limpo['Município'].ffill()
    df_renda_limpo = df_renda_limpo[
        ~(df_renda_limpo['Município'] == df_renda_limpo['Município'].str.upper())
    ].copy()
    df_renda_limpo['Município'] = df_renda_limpo['Município'].str.strip()
    df_renda_limpo['Grupo_Idade'] = df_renda_limpo['Grupo_Idade'].str.strip()
    cols_numericas_renda = ['Total', 'Branca', 'Preta', 'Amarela', 'Parda', 'Indígena']
    df_renda_limpo[cols_numericas_renda] = df_renda_limpo[cols_numericas_renda].replace('-', 0)
    df_renda_limpo[cols_numericas_renda] = df_renda_limpo[cols_numericas_renda].apply(pd.to_numeric, errors='coerce').fillna(0)
    df_renda_limpo = df_renda_limpo.dropna(subset=['Município'])
    dataframes_finais['Renda'] = df_renda_limpo.copy()
    print("-> SUCESSO: Arquivo 3 (Renda) limpo.")


    # --- ARQUIVO 4: ESTADO CIVIL (Tabela 10185) ---
    print("\n[4/10] Processando Arquivo 4: Estado Civil (Tabela 10185)...")
    caminho_civil = '04_IBGE_Censo2022_EstadoCivil_Municipios.xlsx'
    df_civil_limpo = pd.read_excel(caminho_civil, header=3)
    df_civil_limpo.rename(columns={
        'Unnamed: 0': 'Município', 'Unnamed: 1': 'Variável', 'Unnamed: 2': 'Grupo_Idade',
        'Unnamed: 3': 'Cor_Raca', 'Total': 'Total_Pessoas',
        'Viviam em união': 'Casados_Uniao', 'Não viviam em união': 'Nao_Uniao'
    }, inplace=True)
    df_civil_limpo['Município'] = df_civil_limpo['Município'].ffill()
    df_civil_limpo['Grupo_Idade'] = df_civil_limpo['Grupo_Idade'].ffill()
    df_civil_limpo = df_civil_limpo.drop(0).reset_index(drop=True)
    df_civil_limpo['Município'] = df_civil_limpo['Município'].str.strip()
    df_civil_limpo['Grupo_Idade'] = df_civil_limpo['Grupo_Idade'].str.strip()
    df_civil_limpo['Cor_Raca'] = df_civil_limpo['Cor_Raca'].str.strip()
    cols_numericas_civil = ['Total_Pessoas', 'Casados_Uniao', 'Nao_Uniao']
    df_civil_limpo[cols_numericas_civil] = df_civil_limpo[cols_numericas_civil].replace('-', 0)
    df_civil_limpo[cols_numericas_civil] = df_civil_limpo[cols_numericas_civil].apply(pd.to_numeric, errors='coerce').fillna(0)
    df_final_civil = df_civil_limpo.drop(columns=['Variável'])
    dataframes_finais['Estado_Civil'] = df_final_civil.copy()
    print("-> SUCESSO: Arquivo 4 (Estado Civil) limpo.")


    # --- ARQUIVO 5: ESCOLARIDADE (Tabela 10061 - 6 arquivos) ---
    print("\n[5/10] Processando Arquivo 5: Escolaridade (Tabela 10061)...")
    arquivos_escola = [
        '05_A_Escolaridade_Branca.xlsx', '05_B_Escolaridade_Preta.xlsx',
        '05_C_Escolaridade_Parda.xlsx', '05_D_Escolaridade_Amarela.xlsx',
        '05_E_Escolaridade_Indigena.xlsx', '05_F_Escolaridade_Total.xlsx'
    ]
    lista_dfs = []
    for arquivo in arquivos_escola:
        print(f"   - Processando: {arquivo}...")
        linha_munic = detectar_header_municipio(arquivo, pd)
        df_raw = pd.read_excel(arquivo, header=[linha_munic, linha_munic+2, linha_munic+3])
        df_raw.columns = flatten_ibge_cols(df_raw.columns)
        col_munic = [c for c in df_raw.columns if "munic" in c.lower()][0]
        df_raw = df_raw.rename(columns={col_munic: "Município"})
        df_limpo = df_raw.drop(0).reset_index(drop=True)
        if "Branca" in arquivo: r = "Branca"
        elif "Preta" in arquivo: r = "Preta"
        elif "Parda" in arquivo: r = "Parda"
        elif "Amarela" in arquivo: r = "Amarela"
        elif "Indigena" in arquivo: r = "Indígena"
        else: r = "Total"
        df_limpo["Cor_Raca_Arquivo"] = r
        lista_dfs.append(df_limpo)

    df_final_escolaridade = pd.concat(lista_dfs, ignore_index=True)
    df_final_escolaridade['Município'] = df_final_escolaridade['Município'].str.strip()
    cols_numericas = df_final_escolaridade.columns.drop(['Município', 'Sexo', 'Cor_Raca_Arquivo'])
    df_final_escolaridade[cols_numericas] = df_final_escolaridade[cols_numericas].replace('-', 0)
    df_final_escolaridade[cols_numericas] = df_final_escolaridade[cols_numericas].apply(pd.to_numeric, errors='coerce').fillna(0)
    dataframes_finais['Escolaridade'] = df_final_escolaridade.copy()
    print("-> SUCESSO: Arquivo 5 (Escolaridade) limpo e juntado.")


    # --- ARQUIVO 6: SITUAÇÃO (Tabela 9922) ---
    print("\n[6/10] Processando Arquivo 6: Situação (Tabela 9922)...")
    caminho_situacao = '06_IBGE_Censo2022_Situacao_Domicilio_Municipios.xlsx'
    df_situacao = pd.read_excel(caminho_situacao, header=4)
    df_situacao.rename(columns={'Unnamed: 0': 'Município'}, inplace=True)
    df_situacao['Município'] = df_situacao['Município'].str.strip()
    cols_num = ['Total', 'Urbana', 'Rural']
    df_situacao[cols_num] = df_situacao[cols_num].replace('-', 0).apply(pd.to_numeric, errors='coerce').fillna(0)
    dataframes_finais['Situacao'] = df_situacao.copy()
    print("-> SUCESSO: Arquivo 6 (Situação) limpo.")


    # --- ARQUIVO 7: ESGOTO (Tabela 6805) ---
    print("\n[7/10] Processando Arquivo 7: Esgoto (Tabela 6805)...")
    caminho_esgoto = '07_IBGE_Censo2022_Saneamento_Esgoto_Municipios.xlsx'
    df_esgoto_bruto = pd.read_excel(caminho_esgoto, header=[1, 3])
    df_esgoto_bruto.columns = flatten_ibge_cols(df_esgoto_bruto.columns)
    df_esgoto_bruto.rename(columns={df_esgoto_bruto.columns[0]: 'Município'}, inplace=True)
    df_esgoto_limpo = df_esgoto_bruto.drop(0).reset_index(drop=True)
    df_esgoto_limpo['Município'] = df_esgoto_limpo['Município'].str.strip()
    cols_numericas = df_esgoto_limpo.columns[1:]
    df_esgoto_limpo[cols_numericas] = df_esgoto_limpo[cols_numericas].replace('-', 0)
    df_esgoto_limpo[cols_numericas] = df_esgoto_limpo[cols_numericas].apply(pd.to_numeric, errors='coerce').fillna(0)
    dataframes_finais['Esgoto'] = df_esgoto_limpo.copy()
    print("-> SUCESSO: Arquivo 7 (Esgoto) limpo.")


    # --- ARQUIVO 8: ÁGUA (Tabela 6803) ---
    print("\n[8/10] Processando Arquivo 8: Água (Tabela 6803)...")
    caminho_agua = '08_IBGE_Censo2022_Saneamento_Agua_Municipios.xlsx'
    df_agua_limpo = pd.read_excel(caminho_agua, header=3)
    df_agua_limpo.rename(columns={'Unnamed: 0': 'Município'}, inplace=True)
    df_agua_limpo = df_agua_limpo.drop(0).reset_index(drop=True)
    df_agua_limpo['Município'] = df_agua_limpo['Município'].str.strip()
    cols_numericas_agua = df_agua_limpo.columns[1:]
    df_agua_limpo[cols_numericas_agua] = df_agua_limpo[cols_numericas_agua].replace('-', 0)
    df_agua_limpo[cols_numericas_agua] = df_agua_limpo[cols_numericas_agua].apply(pd.to_numeric, errors='coerce').fillna(0)
    dataframes_finais['Agua'] = df_agua_limpo.copy()
    print("-> SUCESSO: Arquivo 8 (Água) limpo.")


    # --- ARQUIVO 9: LIXO (Tabela 6892) ---
    print("\n[9/10] Processando Arquivo 9: Lixo (Tabela 6892)...")
    caminho_lixo = '09_IBGE_Censo2022_Saneamento_Lixo_Municipios.xlsx'
    df_lixo_bruto = pd.read_excel(caminho_lixo, header=3)
    df_lixo_bruto.rename(columns={'Unnamed: 0': 'Município'}, inplace=True)
    df_lixo_limpo = df_lixo_bruto.drop(0).reset_index(drop=True)
    df_lixo_limpo['Município'] = df_lixo_limpo['Município'].str.strip()
    cols_numericas = df_lixo_limpo.columns[1:]
    df_lixo_limpo[cols_numericas] = df_lixo_limpo[cols_numericas].replace('-', 0)
    df_lixo_limpo[cols_numericas] = df_lixo_limpo[cols_numericas].apply(pd.to_numeric, errors='coerce').fillna(0)
    dataframes_finais['Lixo'] = df_lixo_limpo.copy()
    print("-> SUCESSO: Arquivo 9 (Lixo) limpo.")


    # --- ARQUIVO 10: CONSTRUÇÃO (Tabela 9928) ---
    print("\n[10/10] Processando Arquivo 10: Construção (Tabela 9928)...")
    caminho_construcao = '10_IBGE_Censo2022_Moradia_Construcao_Municipios.xlsx'
    df_construcao_limpo = pd.read_excel(caminho_construcao, header=4)
    df_construcao_limpo.rename(columns={'Unnamed: 0': 'Município'}, inplace=True)
    df_construcao_limpo['Município'] = df_construcao_limpo['Município'].str.strip()
    df_construcao_limpo = df_construcao_limpo.loc[:, ~df_construcao_limpo.columns.str.contains('^Unnamed')]
    cols_numericas_const = df_construcao_limpo.columns.drop('Município')
    df_construcao_limpo[cols_numericas_const] = df_construcao_limpo[cols_numericas_const].replace('-', 0)
    df_construcao_limpo[cols_numericas_const] = df_construcao_limpo[cols_numericas_const].apply(pd.to_numeric, errors='coerce').fillna(0)
    df_construcao_limpo = df_construcao_limpo[df_construcao_limpo['Município'].notna()].reset_index(drop=True)
    dataframes_finais['Construcao'] = df_construcao_limpo.copy()
    print("-> SUCESSO: Arquivo 10 (Construção) limpo.")


    # --- FASE 4: A JUNÇÃO FINAL (MERGE) ---

    print("\n\n--- INICIANDO FASE 4: JUNÇÃO (MERGE) ---")

    # 1. Criar Chave_Municipio limpa (sem UF) em todas as tabelas
    print("Criando chaves de junção limpas (sem UF)...")

    # Tabelas que têm o (UF) no nome
    tabelas_com_uf = ['Idade', 'Renda', 'Estado_Civil', 'Escolaridade']
    for nome in tabelas_com_uf:
        dataframes_finais[nome]['Chave_Municipio'] = dataframes_finais[nome]['Município'].str.split(' \(').str[0].str.strip()

    # Tabelas que NÃO têm o (UF) no nome
    tabelas_sem_uf = ['Domicilios_Agg', 'Domicilios_Completo', 'Situacao', 'Esgoto', 'Agua', 'Lixo', 'Construcao']
    for nome in tabelas_sem_uf:
        # Domicilios_Completo não está no merge principal, mas limpamos a chave mesmo assim
        if nome in dataframes_finais:
            dataframes_finais[nome]['Chave_Municipio'] = dataframes_finais[nome]['Município'].str.strip()

    # 2. Filtrar as tabelas complexas para o merge
    print("Agregando tabelas complexas (Renda, Civil, Escolaridade)...")

    df_renda_agg = dataframes_finais['Renda'][
        dataframes_finais['Renda']['Grupo_Idade'] == 'Total'
    ].drop_duplicates(subset=['Chave_Municipio'], keep='first')

    df_civil_agg = dataframes_finais['Estado_Civil'][
        (dataframes_finais['Estado_Civil']['Grupo_Idade'] == 'Total') &
        (dataframes_finais['Estado_Civil']['Cor_Raca'] == 'Total')
    ].drop_duplicates(subset=['Chave_Municipio'], keep='first')

    df_escola_agg = dataframes_finais['Escolaridade'][
        (dataframes_finais['Escolaridade']['Cor_Raca_Arquivo'] == 'Total') &
        (dataframes_finais['Escolaridade']['Sexo'] == 'Total')
    ].drop_duplicates(subset=['Chave_Municipio'], keep='first')

    # 3. JUNTAR (MERGE) TUDO!
    print("Iniciando a junção (merge) das 10 tabelas...")

    # Começa com a tabela de IDADE (nossa base)
    # 🔥 CORREÇÃO ANTI-DUPLICAÇÃO (de 95k para 5570)
    df_dashboard = dataframes_finais['Idade'].drop_duplicates(subset=['Chave_Municipio'], keep='first').copy()

    # Lista das outras tabelas para juntar
    dataframes_para_juntar = {
        'Domicilios': dataframes_finais['Domicilios_Agg'],
        'Renda': df_renda_agg,
        'Estado_Civil': df_civil_agg,
        'Escolaridade': df_escola_agg,
        'Situacao': dataframes_finais['Situacao'],
        'Esgoto': dataframes_finais['Esgoto'],
        'Agua': dataframes_finais['Agua'],
        'Lixo': dataframes_finais['Lixo'],
        'Construcao': dataframes_finais['Construcao']
    }

    # 🔥 CORREÇÃO ANTI-COLISÃO DE NOMES (Erro 'Total_x')

    print("-> Juntando 'Domicilios'...")
    df_dashboard = pd.merge(df_dashboard, dataframes_para_juntar['Domicilios'].drop(columns=['Município'], errors='ignore').drop_duplicates(subset=['Chave_Municipio']), on='Chave_Municipio', how='left')

    print("-> Juntando 'Renda'...")
    df_dashboard = pd.merge(df_dashboard, dataframes_para_juntar['Renda'].drop(columns=['Município'], errors='ignore').drop_duplicates(subset=['Chave_Municipio']), on='Chave_Municipio', how='left', suffixes=('', '_Renda'))

    print("-> Juntando 'Estado_Civil'...")
    df_dashboard = pd.merge(df_dashboard, dataframes_para_juntar['Estado_Civil'].drop(columns=['Município'], errors='ignore').drop_duplicates(subset=['Chave_Municipio']), on='Chave_Municipio', how='left', suffixes=('', '_Civil'))

    print("-> Juntando 'Escolaridade'...")
    df_dashboard = pd.merge(df_dashboard, dataframes_para_juntar['Escolaridade'].drop(columns=['Município'], errors='ignore').drop_duplicates(subset=['Chave_Municipio']), on='Chave_Municipio', how='left', suffixes=('', '_Escola'))

    print("-> Juntando 'Situacao'...")
    df_dashboard = pd.merge(df_dashboard, dataframes_para_juntar['Situacao'].drop(columns=['Município'], errors='ignore').drop_duplicates(subset=['Chave_Municipio']), on='Chave_Municipio', how='left', suffixes=('', '_Situacao'))

    print("-> Juntando 'Esgoto'...")
    df_dashboard = pd.merge(df_dashboard, dataframes_para_juntar['Esgoto'].drop(columns=['Município'], errors='ignore').drop_duplicates(subset=['Chave_Municipio']), on='Chave_Municipio', how='left', suffixes=('', '_Esgoto'))

    print("-> Juntando 'Agua'...")
    df_dashboard = pd.merge(df_dashboard, dataframes_para_juntar['Agua'].drop(columns=['Município'], errors='ignore').drop_duplicates(subset=['Chave_Municipio']), on='Chave_Municipio', how='left', suffixes=('', '_Agua'))

    print("-> Juntando 'Lixo'...")
    df_dashboard = pd.merge(df_dashboard, dataframes_para_juntar['Lixo'].drop(columns=['Município'], errors='ignore').drop_duplicates(subset=['Chave_Municipio']), on='Chave_Municipio', how='left', suffixes=('', '_Lixo'))

    print("-> Juntando 'Construcao'...")
    df_dashboard = pd.merge(df_dashboard, dataframes_para_juntar['Construcao'].drop(columns=['Município'], errors='ignore').drop_duplicates(subset=['Chave_Municipio']), on='Chave_Municipio', how='left', suffixes=('', '_Construcao'))

    # 4. Salvar o arquivo final!
    nome_arquivo_final = '04_DADOS_MESTRES_CENSO_2022_v2.csv'

    # (Usando ';' como separador e ',' como decimal, ideal para Excel em português)
    df_dashboard.to_csv(nome_arquivo_final, index=False, sep=';', decimal=',')

    print("\n--- 🚀 SUCESSO! PROJETO CONCLUÍDO! 🚀 ---")
    print(f"O arquivo final '{nome_arquivo_final}' foi salvo no seu ambiente Colab.")
    print(f"\nTotal de Municípios no arquivo final: {len(df_dashboard)}") # <-- Este número deve ser 5570

    print("\n--- Amostra da Tabela Mestre Final (Corrigida) ---")
    print(df_dashboard.head())

except FileNotFoundError as e:
    print(f"\n--- ERRO FATAL: ARQUIVO NÃO ENCONTRADO ---")
    print(f"Erro: {e}")
    print("Por favor, verifique se você fez o upload de TODOS os 16 arquivos .xlsx para o Colab.")
except Exception as e:
    print(f"\n--- ERRO INESPERADO NO PIPELINE ---")
    print(f"Erro: {e}")

<>:285: SyntaxWarning: invalid escape sequence '\('
<>:285: SyntaxWarning: invalid escape sequence '\('
/tmp/ipython-input-3329451025.py:285: SyntaxWarning: invalid escape sequence '\('
  dataframes_finais[nome]['Chave_Municipio'] = dataframes_finais[nome]['Município'].str.split(' \(').str[0].str.strip()


Iniciando o Pipeline de Limpeza Completo...
Serão processados 10 arquivos-fonte (16 arquivos físicos).
Isto pode demorar alguns minutos...

[1/10] Processando Arquivo 1: Idade (Tabela 9514)...
-> SUCESSO: Arquivo 1 (Idade) limpo.

[2/10] Processando Arquivo 2: Domicílios (Tabela 9879)...
-> SUCESSO: Arquivo 2 (Domicílios) limpo.

[3/10] Processando Arquivo 3: Renda (Tabela 10291)...
-> SUCESSO: Arquivo 3 (Renda) limpo.

[4/10] Processando Arquivo 4: Estado Civil (Tabela 10185)...
-> SUCESSO: Arquivo 4 (Estado Civil) limpo.

[5/10] Processando Arquivo 5: Escolaridade (Tabela 10061)...
   - Processando: 05_A_Escolaridade_Branca.xlsx...
   - Processando: 05_B_Escolaridade_Preta.xlsx...
   - Processando: 05_C_Escolaridade_Parda.xlsx...
   - Processando: 05_D_Escolaridade_Amarela.xlsx...
   - Processando: 05_E_Escolaridade_Indigena.xlsx...
   - Processando: 05_F_Escolaridade_Total.xlsx...
-> SUCESSO: Arquivo 5 (Escolaridade) limpo e juntado.

[6/10] Processando Arquivo 6: Situação (Tabela 9